## Model Experiments

Ahora llevaremos a cabo una serie de experimentos para entrenar y evaluar los modelos para clasificación de texto. Y elegiremos cuál es el mejor y los loggearemos a MLflow

Importamos librerías

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

import warnings
warnings.filterwarnings('ignore')
import pandas as pd

Configuramos MLflow con DagsHub para rastrear y registrar nuestros experimentos.

In [2]:
import dagshub
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)

Accessing as JuanPab2009

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

Cargamos los datos procesados

In [4]:
df = pd.read_csv('../data/processed_data/processed_tickets.csv')

df.head()

,complaint_what_happened,ticket_classification
0,good morning name appreciate could help put st...,Debt collection + Credit card debt
1,upgraded card //2018 told agent upgrade annive...,Credit card or prepaid card + General-purpose ...
2,"//2018 , trying book ticket , came across offe...","Credit reporting, credit repair services, or o..."
3,grand son give check { $ 1600.00 } deposit cha...,Checking or savings account + Checking account
4,please remove inquiry,"Credit reporting, credit repair services, or o..."


Seleccionamos las columnas que utilizaremos como variables independientes (X) y la variable dependiente (y) para el modelado.

In [5]:
X = df['complaint_what_happened']
y = df['ticket_classification']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Ahora vamos a convertir el texto en una representación númerica, vectorizando con TF-IDF. Esto nos va a permitir alimentar los datos textuales a los modelos.

In [6]:
# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar y transformar los datos de entrenamiento
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transformar los datos de prueba
X_test_tfidf = tfidf_vectorizer.transform(X_test)


Ahora haremos nuestro primer experimento

In [7]:
mlflow.set_experiment('juan-blanco-logistic-regression')


2024/11/20 18:56:49 INFO mlflow.tracking.fluent: Experiment with name 'juan-blanco-logistic-regression' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/33a06b90f9704c8da6f10c2af583fb9e', creation_time=1732150609288, experiment_id='9', last_update_time=1732150609288, lifecycle_stage='active', name='juan-blanco-logistic-regression', tags={}>

### Modelo base logistic regression

Primero vamos a entrenar un modelo básico de Regresión Logística y registraremos sus métricas y artefactos a MLflow.

In [8]:
with mlflow.start_run(run_name='LogisticRegression_Baseline'):
    lr_model = LogisticRegression(max_iter=1000)
    
    # Entrenar el modelo
    lr_model.fit(X_train_tfidf, y_train)
    
    # Hacer predicciones
    y_pred = lr_model.predict(X_test_tfidf)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Registrar métrica
    mlflow.log_metric('accuracy', accuracy)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(lr_model, 'logistic_regression_model')
    
    # Imprimir resultados
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))


2024/11/20 18:57:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 0.6144678492239468
                                                                                                 precision    recall  f1-score   support

                                                     Bank account or service + Checking account       0.66      0.22      0.34       249
                                           Bank account or service + Other bank product/service       0.00      0.00      0.00        60
                                                 Checking or savings account + Checking account       0.60      0.89      0.72       758
                                 Checking or savings account + Other banking product or service       0.00      0.00      0.00        48
                                                  Checking or savings account + Savings account       0.60      0.13      0.21        46
                                                                   Consumer Loan + Vehicle loan       0.67      0.06      0.11        35
           

Obtuvimos un Accuracy de 0.6144 que es bastante bueno para una regresión logística básica y tardó relativamente muy poco.

### Modelo con Hyperparameter Tuning de Regresión Logística

Ahora vamos a hacer una Regresión Logística con una busqueda de hiperparámetros utilizando GridSearchCV para poder optimizar el rendimiento del modelo.

In [9]:
# Hiperparámetros a probar
param_grid_lr = {
    'C': [0.1, 1, 10],          # Parámetro de regularización inversa
    'penalty': ['l1', 'l2'],    # Tipos de penalización
    'solver': ['liblinear']     # Solvers que soportan penalización L1
}

# Configurar GridSearchCV
grid_search_lr = GridSearchCV(
    LogisticRegression(max_iter=1000),
    param_grid_lr,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

Ahora vamos a entrenar el modelo con diferentes combinaciones de hiperparámetros, identificamos el mejor modelo y lo loggeamos.

In [10]:
# Iniciar un run de mlflow
with mlflow.start_run(run_name='LogisticRegression_Tuning'):
    # Entrenar con diferentes hiperparámetros
    grid_search_lr.fit(X_train_tfidf, y_train)
    
    # Mejor modelo
    best_lr_model = grid_search_lr.best_estimator_
    
    # Hacer predicciones
    y_pred = best_lr_model.predict(X_test_tfidf)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Registrar métricas y parámetros
    mlflow.log_params(grid_search_lr.best_params_)
    mlflow.log_metric('accuracy', accuracy)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(best_lr_model, 'best_logistic_regression_model')
    
    # Imprimir resultados
    print(f"Best Params: {grid_search_lr.best_params_}")
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))


2024/11/20 19:15:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best Params: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy: 0.6122505543237251
                                                                                                 precision    recall  f1-score   support

                                                     Bank account or service + Checking account       0.49      0.31      0.38       249
                                           Bank account or service + Other bank product/service       0.25      0.05      0.08        60
                                                 Checking or savings account + Checking account       0.63      0.82      0.71       758
                                 Checking or savings account + Other banking product or service       0.14      0.02      0.04        48
                                                  Checking or savings account + Savings account       0.42      0.22      0.29        46
                                                                   Consumer Loan + Ve

Optuvimos un accuracy ligeramente inferior al modelo base de regresión logística, y tardó más, por lo que este modelo lo podemos descartar de simple vista para nuestro champion.

### Modelo SVC

Ahora haremos un modelo básico de Suport Vector Machine, este modelo suele ser bueno en problemas de clasificación ya que maneja bien espacios de alta dimensión, como los datos vectorizados con TF-IDF, por lo que debería de darnos un buen accuracy.

In [11]:
mlflow.set_experiment('juan-blanco-svc')

2024/11/20 19:25:27 INFO mlflow.tracking.fluent: Experiment with name 'juan-blanco-svc' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/ca087f019bc746c09e3042068090aa20', creation_time=1732152327465, experiment_id='10', last_update_time=1732152327465, lifecycle_stage='active', name='juan-blanco-svc', tags={}>

In [12]:
# Iniciar un run de mlflow
with mlflow.start_run(run_name='SVC_Baseline'):
    # Crear el modelo
    svc_model = SVC()
    
    # Entrenar el modelo
    svc_model.fit(X_train_tfidf, y_train)
    
    # Hacer predicciones
    y_pred = svc_model.predict(X_test_tfidf)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Registrar métrica
    mlflow.log_metric('accuracy', accuracy)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(svc_model, 'svc_model')
    
    # Imprimir resultados
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))


2024/11/20 19:27:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 0.6202882483370288
                                                                                                 precision    recall  f1-score   support

                                                     Bank account or service + Checking account       0.86      0.18      0.29       249
                                           Bank account or service + Other bank product/service       0.00      0.00      0.00        60
                                                 Checking or savings account + Checking account       0.59      0.91      0.71       758
                                 Checking or savings account + Other banking product or service       0.00      0.00      0.00        48
                                                  Checking or savings account + Savings account       0.71      0.11      0.19        46
                                                                   Consumer Loan + Vehicle loan       0.67      0.06      0.11        35
           

Ok nos dio un accuracy ligeramente mejor con 0.62 y no tardó tanto, este puede ser un buen candidato a ser champion, vamos a ver qué tanto cambia con una tuneada de hyperparametros.

### Hyperparameter Tuning para SVC

Ahora realizaremos una búsqueda de hiperparámetros utilizando de nuevo GridSearchCV, veamos si funciona mejor.

In [13]:
# Hiperparámetros a probar
param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],  # Tipos de kernel
    'gamma': ['scale', 'auto']
}

# Configurar GridSearchCV
grid_search_svc = GridSearchCV(
    SVC(),
    param_grid_svc,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

In [15]:
with mlflow.start_run(run_name='SVC_Tuning'):
    # Entrenar con diferentes hiperparámetros
    grid_search_svc.fit(X_train_tfidf, y_train)
    
    # Mejor modelo
    best_svc_model = grid_search_svc.best_estimator_
    
    # Hacer predicciones
    y_pred = best_svc_model.predict(X_test_tfidf)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Registrar métricas y parámetros
    mlflow.log_params(grid_search_svc.best_params_)
    mlflow.log_metric('accuracy', accuracy)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(best_svc_model, 'best_svc_model')
    
    # Imprimir resultados
    print(f"Best Params: {grid_search_svc.best_params_}")
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))

2024/11/20 19:54:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best Params: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.6322062084257206
                                                                                                 precision    recall  f1-score   support

                                                     Bank account or service + Checking account       0.67      0.30      0.42       249
                                           Bank account or service + Other bank product/service       0.33      0.02      0.03        60
                                                 Checking or savings account + Checking account       0.62      0.87      0.73       758
                                 Checking or savings account + Other banking product or service       0.00      0.00      0.00        48
                                                  Checking or savings account + Savings account       0.50      0.20      0.28        46
                                                                   Consumer Loan + Vehicle

Un muy buen accuracy, subió a 0.63 pero tardó considerablemente más.

Ahora cómo definimos cuál va a ser el champion y cuál va a ser el challenger, aunque las métricas no son tan distantes y el tiempo fue similar (a excepción del SVC tuneado), considero que en este caso si me ire por el que mejor accuracy tenga. 

El tiempo de entrenamiento si es  ligeramente superior, sin embargo, no es un tiempo tan descabellado por lo que considero que sería la mejor opción. Así que vamos a hacer una función que defina a nuestro champion y nuestro challenger teniendo en cuenta el accuracy.

In [ ]:
mlflow.set_tracking_uri('https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow')

client = MlflowClient()

In [4]:
experiment_name = "juan-blanco-svc"

# Obtener el experimento por su nombre
experiment = client.get_experiment_by_name(experiment_name)

# Verificar que el experimento existe
if experiment is None:
    print(f"El experimento '{experiment_name}' no se encontró.")
else:
    # Obtener el ID del experimento
    experiment_id = experiment.experiment_id

    # Buscar los dos mejores runs en base al accuracy
    top_runs = mlflow.search_runs(
        experiment_ids=[experiment_id],
        order_by=["metrics.accuracy DESC"],
        max_results=2
    )

    # Verificar que tenemos al menos dos runs
    if len(top_runs) < 2:
        print("No se encontraron suficientes runs para seleccionar Champion y Challenger.")
    else:
        # Obtenemos los IDs de los mejores runs
        champion_run = top_runs.iloc[0]
        challenger_run = top_runs.iloc[1]

        # Obtenemos los IDs de las ejecuciones
        champion_run_id = champion_run.run_id
        challenger_run_id = challenger_run.run_id

        # Obtenemos las URIs de los modelos
        champion_model_uri = f"runs:/{champion_run_id}/model"
        challenger_model_uri = f"runs:/{challenger_run_id}/model"

        # Declaramos el nombre del modelo registrado
        model_name = "blanco-model"

        # Registrar el modelo Champion
        champion_model_version = mlflow.register_model(
            model_uri=champion_model_uri,
            name=model_name
        )
        client.set_registered_model_alias(
            name=model_name,
            alias='champion',
            version=champion_model_version.version
        )

        # Registrar el modelo Challenger
        challenger_model_version = mlflow.register_model(
            model_uri=challenger_model_uri,
            name=model_name
        )
        client.set_registered_model_alias(
            name=model_name,
            alias='challenger',
            version=challenger_model_version.version
        )

        print(f"Champion Model: Run ID {champion_run_id}, Version {champion_model_version.version}")
        print(f"Challenger Model: Run ID {challenger_run_id}, Version {challenger_model_version.version}")

Registered model 'blanco-model' already exists. Creating a new version of this model...
2024/11/21 13:11:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: blanco-model, version 2
Created version '2' of model 'blanco-model'.
Registered model 'blanco-model' already exists. Creating a new version of this model...
2024/11/21 13:11:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: blanco-model, version 3
Created version '3' of model 'blanco-model'.


Champion Model: Run ID 72c43d927dce4607a0cae9a3e07eacd6, Version 2
Challenger Model: Run ID ca7f19ef30b84c2790a744cf2a3dc389, Version 3


Listo, ya quedaron registrados nuestros modelos champion y challenger que ambos fueron SVM, uno tuneado y el otro no.